# Projet : Modèle Transformer pour la Médecine Reproductive

### Objectif du projet

Ce projet a pour objectif de développer un modèle de **Machine Learning** basé sur un Transformer, tel que **BERT** ou un modèle personnalisé, pour traiter des données textuelles relatives à la médecine reproductive. Le dataset utilisé est composé de 50 fichiers JSON provenant d'articles médicaux.

Les principales étapes de ce projet sont les suivantes :
- Télécharger et charger les données.
- Prétraiter les données textuelles pour les rendre compatibles avec un modèle Transformer.
- Concevoir et entraîner un modèle Transformer pour une tâche spécifique.
- Évaluer le modèle avec des métriques de performance.
- Documenter chaque étape du processus dans ce notebook.

---

### Structure du Notebook

Ce notebook est organisé comme suit :
1. **Téléchargement et chargement des données**
2. **Prétraitement des données**
3. **Construction et entraînement du modèle Transformer**
4. **Évaluation du modèle**
5. **Conclusion et perspectives**

L'objectif est d'illustrer chaque étape avec des explications détaillées et d'exécuter le code correspondant.

---



In [1]:
!pip install pandas transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


### 1. Télécharger et Charger les Données

Avant de commencer à manipuler les données, il est nécessaire de les charger dans l'environnement de travail. Supposons que le fichier ZIP contenant les fichiers JSON a déjà été téléchargé sur Google Colab. Nous allons commencer par extraire ce fichier ZIP et charger les données dans une liste pour les traiter par la suite.

---

#### 1.1 Extraction du Fichier ZIP

Nous allons d'abord extraire le fichier ZIP qui contient les fichiers JSON. Cette étape est nécessaire pour avoir accès aux données sous forme de fichiers individuels.

**Code :**



In [2]:
import zipfile
import os

# Spécifiez le chemin vers votre fichier .zip
zip_path = "assignementdataset.zip"
extraction_path = "Data_set"

# Extraction
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

print(f"Fichiers extraits dans : {extraction_path}")


Fichiers extraits dans : Data_set


**Explication :**

Le code ci-dessus extrait le contenu du fichier ZIP dans le répertoire Data_set. Cette opération nous permet d'accéder aux fichiers JSON à l'intérieur du ZIP.

1.2 Chargement des Fichiers JSON

Une fois les fichiers extraits, nous allons parcourir chaque fichier JSON, extraire les données pertinentes (titre, résumé, texte) et les organiser dans une structure adaptée.

2. Prétraitement des Données :

Une fois les données chargées, il est important de les préparer avant de les passer dans le modèle Transformer. Le prétraitement peut inclure la tokenisation, la gestion de la longueur des séquences et l'encodage des labels.



In [3]:
import os
import json
import pandas as pd

# Define categories
categories = {
    "Procedures": 0,
    "Outcomes": 1,
    "Technologies": 2,
    "General Research": 3
}

# Path to extracted JSON files
data_dir = '/content/Data_set/assignementdataset'  # Remplacez par votre chemin de répertoire
data = []

# Fonction pour assigner une catégorie en fonction du titre
def assign_category_based_on_title(title):
    # Mots-clés pour chaque catégorie
    procedures_keywords = ["procedure", "treatment", "method", "surgical", "intervention"]
    outcomes_keywords = ["outcome", "effect", "result", "consequence", "impact"]
    technologies_keywords = ["technology", "diagnostic", "device", "test", "machine"]

    # Assigner la catégorie en fonction de la présence de mots-clés dans le titre
    title_lower = title.lower()
    if any(keyword in title_lower for keyword in procedures_keywords):
        return categories["Procedures"]
    elif any(keyword in title_lower for keyword in outcomes_keywords):
        return categories["Outcomes"]
    elif any(keyword in title_lower for keyword in technologies_keywords):
        return categories["Technologies"]
    else:
        return categories["General Research"]

# Parse JSON files
for file_name in os.listdir(data_dir):
    if file_name.endswith(".json"):
        with open(os.path.join(data_dir, file_name), 'r') as f:
            content = json.load(f)
            title = content.get("title", "")
            abstract = " ".join([item["text"] for item in content.get("pdf_parse", {}).get("abstract", [])])
            body_text = " ".join([item["text"] for item in content.get("pdf_parse", {}).get("body_text", [])])
            text = title + " " + abstract + " " + body_text

            # Assigner la catégorie basée sur le titre
            label = assign_category_based_on_title(title)
            data.append({"text": text, "label": label})

# Convert to DataFrame
df = pd.DataFrame(data)
print(df.head())


                                                text  label
0  Assisted reproductive technology: consideratio...      2
1  Point of care rapid test for diagnosis of syph...      2
2  Peri-implantation glucocorticoid administratio...      2
3  Interventions for uterine fibroids: an overvie...      0
4  Antimüllerian hormone is not associated with e...      2


**Explication :**
Ici, nous chargeons les fichiers JSON extraits et récupérons les informations nécessaires (titre, résumé, corps de l'article). Ensuite, nous utilisons une fonction pour attribuer une catégorie à chaque article en fonction des mots-clés présents dans le titre. Ces informations sont organisées dans un DataFrame.

2.1  Séparation des Données:




Avant d'entraîner le modèle, nous devons diviser les données en ensembles d'entraînement et de test. Cela permettra de tester le modèle sur des données qu'il n'a pas vues lors de l'entraînement.

In [4]:
from sklearn.model_selection import train_test_split

# Split dataset
train_texts, test_texts = train_test_split(df, test_size=0.2, random_state=42)

print(f"Train size: {len(train_texts)}")
print(f"Test size: {len(test_texts)}")


Train size: 40
Test size: 10


**Explication :**
Nous divisons ici les données en un ensemble d'entraînement (80 %) et un ensemble de test (20 %), ce qui nous permettra de tester la performance du modèle après l'entraînement.

In [5]:
train_texts


,text,label
12,A “first” on the horizon: the expansion of ute...,3
4,Antimüllerian hormone is not associated with e...,2
37,"Ascending toward a 30,000 foot view of uterus ...",3
8,"Awareness, knowledge, and misconceptions of ad...",3
3,Interventions for uterine fibroids: an overvie...,0
6,Routine ultrasound for fetal assessment before...,3
41,A framework approach for hysteroscopic uterine...,3
46,Antimüllerian hormone and leukocyte aging mark...,3
47,Are sperm human papilloma virus infections a m...,3
15,Association of endometriosis and adenomyosis w...,3


In [6]:
test_texts

,text,label
13,Maternal postures for fetal malposition in lab...,3
39,A combination of two novel ligation techniques...,3
30,A behind-the-scenes look at retroperitoneal ec...,3
45,Ultrasound versus 'clinical touch' for cathete...,3
17,Preconception lifestyle advice for people with...,3
48,Perioperative pharmacological interventions fo...,0
26,Monotherapy treatment of epilepsy in pregnancy...,0
25,Adenomyosis: single-cell transcriptomic analys...,3
32,Point of care rapid test for diagnosis of syph...,2
19,Automated identification of rare sperm becomes...,3


2.2 Tokenisation des Textes :

Les modèles Transformer tels que BERT nécessitent des textes sous forme de tokens (sous-mots ou mots). Nous allons donc utiliser un tokenizer pour convertir les textes en tokens et les préparer pour l'entraînement.

**Code :**

In [7]:
from transformers import AutoTokenizer
from datasets import Dataset

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_texts)
test_dataset = Dataset.from_pandas(test_texts)

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

# Apply tokenization
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

**Explication :**
Nous utilisons ici le tokenizer BERT pour transformer les textes en tokens et les préparer pour l'entraînement. La longueur des séquences est limitée à 512 tokens pour être compatible avec le modèle BERT

3. Construction et Entraînement du Modèle Transformer
Une fois les données prétraitées, nous allons construire le modèle Transformer et l'entraîner sur les données d'entraînement.

3.1 Charger le Modèle BERT et Définir les Arguments d'Entraînement
Nous chargeons un modèle pré-entraîné BERT et le configurons pour la classification des textes en 4 catégories.

3.2 Entraîner le Modèle
Nous entraînons maintenant le modèle sur les données d'entraînement.

**Code :**

**Code :**

In [13]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load pre-trained model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    num_train_epochs=180,
    weight_decay=0.01,
    logging_dir="./logs",  # Répertoire pour les journaux
    logging_steps=10,      # Journaliser toutes les 10 étapes
    report_to="none"       # Désactiver les intégrations externes comme W&B
)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)  # Obtenir la classe prédite

    # Calcul des métriques
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics  # Ajout de la fonction de calcul des métriques
)


# Train the model
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.063706,0.700000,0.490000,0.700000,0.576471
2,No log,0.942326,0.700000,0.490000,0.700000,0.576471
3,1.051900,0.903170,0.700000,0.490000,0.700000,0.576471
4,1.051900,0.940132,0.700000,0.490000,0.700000,0.576471
5,0.735600,0.923431,0.700000,0.490000,0.700000,0.576471
6,0.735600,0.920150,0.700000,0.490000,0.700000,0.576471
7,0.735600,0.912795,0.700000,0.490000,0.700000,0.576471
8,0.644800,0.827080,0.700000,0.490000,0.700000,0.576471
9,0.644800,0.889157,0.700000,0.490000,0.700000,0.576471
10,0.490600,0.935420,0.700000,0.490000,0.700000,0.576471


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

TrainOutput(global_step=720, training_loss=0.06005780910685037, metrics={'train_runtime': 751.7797, 'train_samples_per_second': 9.577, 'train_steps_per_second': 0.958, 'total_flos': 1894433616691200.0, 'train_loss': 0.06005780910685037, 'epoch': 180.0})

Explication :
Nous chargeons le modèle BERT et le configurons avec des arguments d'entraînement, tels que le taux d'apprentissage, la taille du batch et le nombre d'époques. Le modèle est prêt pour l'entraînement.



In [11]:
# Evaluate the model
results = trainer.evaluate()
print("Evaluation Results:", results)


Evaluation Results: {'eval_loss': 0.906141459941864, 'eval_accuracy': 0.8, 'eval_precision': 0.6444444444444445, 'eval_recall': 0.8, 'eval_f1': 0.7125, 'eval_runtime': 0.2884, 'eval_samples_per_second': 34.669, 'eval_steps_per_second': 6.934, 'epoch': 180.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
# Save model
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")

# Load and test
from transformers import pipeline

classifier = pipeline("text-classification", model="./trained_model", tokenizer=tokenizer)
test_text = "This study evaluates the efficacy of saline solutions for outpatient procedures."
result = classifier(test_text)
print(result)


Device set to use cuda:0


[{'label': 'LABEL_3', 'score': 0.4845578968524933}]
